# Formula 1 Analysis in SQL 

**[The Dataset:](https://www.kaggle.com/datasets/cjgdev/formula-1-race-data-19502017)**

Formula 1 (officially the FIA Formula One World Championship) is the highest class of single-seat auto racing that is sanctioned by the Fédération Internationale de l'Automobile (FIA). 

This dataset contains data from 1950 all the way through the 2017 season, and consists of tables describing constructors, race drivers, lap times, pit stops and more.


**Data Description**
* **Circuits:** Description about the location of each track that has been included in the Formula 1 circuit since 1950.
* **Constructor Results:** Results based on Constructor (Team)
* **Drivers:** Description about every driver that has every raced in the Formula 1 circuit since 1950
* **Driver Standings:** Driver Standings in each year since 1950
* **Lap Times:** Laptimes of every driver, from every race
* **Pit Stops:** Amount of pitstop and the time the pitstop took
* **Qualifying:** Qualifying position of each racer, from every race
* **Races:** Each race, at every location, winner, fastest lap, winning team
* **Results:** Results from each race
* **Seasons:** Results at the end of each season

# Question & Analysis

***Between 2013 and 2014, a decision was made to manipulate the formula of the cars. Starting in 2014, there would be a new V6 engine with 1600cc / 8 gearbox. Did this affect lap times and if so, which drivers demonstrated the largest impact?***

CC is the displacement volume of the engine, so it means that that the engine has more cylinders and a higher swept volume which directly translates into horse power and torque of the vehicle.

Information: https://bleacherreport.com/articles/2003467-are-2014-formula-1-cars-slower-analysing-lap-times-at-australian-grand-prix




# FINAL RESULT 

final result for lazy readers:
<center>
<img alt="final_result" src="f1res.png" width="600px" />
</center>

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sqlite3
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
np.set_printoptions(threshold=5) # Avoid printing out big matrices
%matplotlib inline


In [22]:
# !pip install ipython-sql

In [23]:
#create a dataframe to import data
import pandas as pd 
#load the data
results = pd.read_csv("./Files/results.csv")
races = pd.read_csv("./Files/races.csv")
drivers = pd.read_csv("./Files/driversutf8.csv")
lapT = pd.read_csv("./Files/lapTimes.csv")

#UTF 
# import chardet
# with open("./Files/races.csv", 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(100000))
# result

races

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
992,1005,2018,17,22,Japanese Grand Prix,2018-10-07,05:00:00,http://en.wikipedia.org/wiki/2018_Japanese_Gra...
993,1006,2018,18,69,United States Grand Prix,2018-10-21,19:00:00,http://en.wikipedia.org/wiki/2018_United_State...
994,1007,2018,19,32,Mexican Grand Prix,2018-10-28,19:00:00,http://en.wikipedia.org/wiki/2018_Mexican_Gran...
995,1008,2018,20,18,Brazilian Grand Prix,2018-11-11,16:00:00,http://en.wikipedia.org/wiki/2018_Brazilian_Gr...


In [24]:
#slwite library connect
cnn = sqlite3.connect('tutorial.db')

results.to_sql('results', cnn, if_exists='replace')
races.to_sql('races', cnn, if_exists='replace')
drivers.to_sql('drivers', cnn, if_exists='replace')
lapT.to_sql('lapT', cnn, if_exists='replace')

In [25]:
#load 
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [26]:
%sql sqlite:///tutorial.db

In [58]:
%%sql

SELECT * FROM results
LIMIT 5;

 * sqlite:///tutorial.db
Done.


index,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


In [28]:
#converting fastestLapTime to seconds

def get_sec(time_str):
    """Get seconds from time."""
    m, s = time_str.split(':')
    s, ms = s.split(".")
    #return m, s, ms
    return (int(m) * 60) + int(s) + (int(ms) * 0.001)

print(get_sec('01:27.5')) # one minute, 27 seconds, 5 milliseconds


87.005


In [29]:
#EDA
enco = results.merge(races, how="inner", on="raceId").drop(columns=["constructorId", "number", "grid", "points", "url", "time_x", "circuitId", "positionOrder", "rank", "statusId", "round"])
#enco.rename(columns={"time_y": "race time", "name" : "race name"})
enco = enco.merge(drivers, how="inner", on="driverId").drop(columns=["dob", "url", "number", "driverRef"])
enco = enco.rename(columns={"time_y": "racetime", "name" : "racename"})
enco.count()#enco[enco["fastestLapTime"] != None]
enco = enco.dropna(subset=['fastestLapTime'])
enco["fastestSeconds"] = enco["fastestLapTime"].apply(lambda x: get_sec(x))
enconew = enco
enconew

,resultId,raceId,driverId,position,positionText,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,year,racename,date,racetime,code,forename,surname,nationality,fastestSeconds
0,1,18,1,1.0,1,58,5690616.0,39.0,01:27.5,218.3,2008,Australian Grand Prix,2008-03-16,04:30:00,HAM,Lewis,Hamilton,British,87.005
1,27,19,1,5.0,5,56,5525103.0,53.0,01:35.5,209.033,2008,Malaysian Grand Prix,2008-03-23,07:00:00,HAM,Lewis,Hamilton,British,95.005
2,57,20,1,13.0,13,56,NaN,25.0,01:35.5,203.969,2008,Bahrain Grand Prix,2008-04-06,11:30:00,HAM,Lewis,Hamilton,British,95.005
3,69,21,1,3.0,3,66,5903238.0,20.0,01:22.0,204.323,2008,Spanish Grand Prix,2008-04-27,12:00:00,HAM,Lewis,Hamilton,British,82.000
4,90,22,1,2.0,2,58,5213230.0,31.0,01:26.5,222.085,2008,Turkish Grand Prix,2008-05-11,12:00:00,HAM,Lewis,Hamilton,British,86.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23772,23777,988,842,16.0,16,54,NaN,33.0,01:43.8,192.542,2017,Abu Dhabi Grand Prix,2017-11-26,13:00:00,GAS,Pierre,Gasly,French,103.008
23773,23714,985,843,13.0,13,55,NaN,47.0,01:40.0,198.509,2017,United States Grand Prix,2017-10-22,19:00:00,HAR,Brendon,Hartley,New Zealander,100.000
23774,23739,986,843,NaN,R,30,NaN,28.0,01:22.6,187.647,2017,Mexican Grand Prix,2017-10-29,19:00:00,HAR,Brendon,Hartley,New Zealander,82.006
23775,23758,987,843,NaN,R,40,NaN,29.0,01:14.7,207.779,2017,Brazilian Grand Prix,2017-11-12,16:00:00,HAR,Brendon,Hartley,New Zealander,74.007


In [30]:
enconew.to_sql('encon', cnn, if_exists='replace') #add table to database

# #analysis of change from eyars 2013 & 2014
#801 rows
query_1 = """
SELECT * FROM enconew
WHERE year == 2013 or year == 2014
;
"""
res_1 = pd.read_sql(query_1, cnn)
res_1

,index,resultId,raceId,driverId,position,positionText,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,year,race name,date,race time,code,forename,surname,nationality,fastestSeconds
0,110,21716,880,1,5.0,5,58,5448786.0,45.0,01:29.8,212.689,2013,Australian Grand Prix,2013-03-17,06:00:00,HAM,Lewis,Hamilton,British,89.008
1,111,21736,881,1,3.0,3,56,5948862.0,32.0,01:41.0,197.57,2013,Malaysian Grand Prix,2013-03-24,08:00:00,HAM,Lewis,Hamilton,British,101.000
2,112,21758,882,1,3.0,3,56,5799267.0,50.0,01:40.0,196.273,2013,Chinese Grand Prix,2013-04-14,07:00:00,HAM,Lewis,Hamilton,British,100.000
3,113,21782,883,1,5.0,5,57,5795728.0,48.0,01:38.2,198.395,2013,Bahrain Grand Prix,2013-04-21,12:00:00,HAM,Lewis,Hamilton,British,98.002
4,114,21811,884,1,12.0,12,65,NaN,53.0,01:27.9,190.659,2013,Spanish Grand Prix,2013-05-12,12:00:00,HAM,Lewis,Hamilton,British,87.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,23346,22414,912,828,20.0,20,51,NaN,47.0,01:30.3,231.001,2014,Italian Grand Prix,2014-09-07,12:00:00,ERI,Marcus,Ericsson,Swedish,90.003
797,23347,22431,913,828,15.0,15,60,7298983.0,14.0,01:55.4,157.985,2014,Singapore Grand Prix,2014-09-21,12:00:00,ERI,Marcus,Ericsson,Swedish,115.004
798,23348,22455,914,828,17.0,17,43,NaN,27.0,01:54.7,182.309,2014,Japanese Grand Prix,2014-10-05,06:00:00,ERI,Marcus,Ericsson,Swedish,114.007
799,23349,22479,915,828,19.0,19,51,NaN,48.0,01:44.0,202.471,2014,Russian Grand Prix,2014-10-12,11:00:00,ERI,Marcus,Ericsson,Swedish,104.000


In [31]:
# #5 drivers with the largest differences in laptime Seconds from 2013 to 2014.

# query_2 = """
# SELECT driverId, MAX(fastestSeconds) - MIN(fastestSeconds) AS swing
# FROM enconew
# WHERE (year == 2013 OR year == 2014) 
# GROUP BY driverId
# ORDER BY swing DESC
# LIMIT 5;

# """
# res_2 = pd.read_sql(query_2, cnn)
# res_2

#INCORRECT not accounting for 2013 versus 2014 ?? just largest difference 

In [34]:
# 2013 drivers

query_3 = """
SELECT raceId AS raceId13, driverId, fastestSeconds AS fastestSeconds13, year AS year13, forename, surname
FROM enconew
WHERE year = '2013'
GROUP BY driverId
;
"""
res_3 = pd.read_sql(query_3, cnn)
res_3

,raceId13,driverId,fastestSeconds13,year13,forename,surname
0,880,1,89.008,2013,Lewis,Hamilton
1,880,3,92.003,2013,Nico,Rosberg
2,880,4,89.006,2013,Fernando,Alonso
3,898,5,101.000,2013,Heikki,Kovalainen
4,880,8,89.003,2013,Kimi,Rج_ikkج¦nen
5,880,13,90.002,2013,Felipe,Massa
6,880,16,90.007,2013,Adrian,Sutil
7,880,17,89.007,2013,Mark,Webber
8,880,18,90.002,2013,Jenson,Button
9,880,20,90.004,2013,Sebastian,Vettel


In [36]:
#2014 drivers

res_3.to_sql('res_3', cnn, if_exists='replace')

query_4 = """
SELECT raceId AS raceId14, driverId, fastestSeconds AS fastestSeconds14, year AS year14, forename, surname
FROM enconew
WHERE year = '2014'
GROUP BY driverId
;
"""
res_4 = pd.read_sql(query_4, cnn)
res_4

,raceId14,driverId,fastestSeconds14,year14,forename,surname
0,900,1,109.009,2014,Lewis,Hamilton
1,900,3,92.005,2014,Nico,Rosberg
2,900,4,93.002,2014,Fernando,Alonso
3,900,8,93.002,2014,Kimi,Rج_ikkج¦nen
4,900,13,100.003,2014,Felipe,Massa
5,900,16,93.004,2014,Adrian,Sutil
6,900,18,92.009,2014,Jenson,Button
7,900,20,97.001,2014,Sebastian,Vettel
8,900,154,95.003,2014,Romain,Grosjean
9,901,155,107.008,2014,Kamui,Kobayashi


In [40]:
#joining 2013 & 2014

res_4.to_sql('res_4', cnn, if_exists='replace')

# 
query_5 = """
SELECT  q4.raceId14 AS raceId14, q4.driverId, q4.fastestSeconds14 AS fastestSeconds14, q4.year14 AS year14, q4.forename, q4.surname, q3.raceId13, q3.year13, q3.fastestSeconds13
FROM res_4 AS q4
JOIN res_3 q3 ON q3.driverId = q4.driverId;
"""
res_5 = pd.read_sql(query_5, cnn)
res_5

,raceId14,driverId,fastestSeconds14,year14,forename,surname,raceId13,year13,fastestSeconds13
0,900,1,109.009,2014,Lewis,Hamilton,880,2013,89.008
1,900,3,92.005,2014,Nico,Rosberg,880,2013,92.003
2,900,4,93.002,2014,Fernando,Alonso,880,2013,89.006
3,900,8,93.002,2014,Kimi,Rج_ikkج¦nen,880,2013,89.003
4,900,13,100.003,2014,Felipe,Massa,880,2013,90.002
5,900,16,93.004,2014,Adrian,Sutil,880,2013,90.007
6,900,18,92.009,2014,Jenson,Button,880,2013,90.002
7,900,20,97.001,2014,Sebastian,Vettel,880,2013,90.004
8,900,154,95.003,2014,Romain,Grosjean,880,2013,90.004
9,900,807,92.006,2014,Nico,Hج_lkenberg,881,2013,100.007


In [46]:

res_5.to_sql('res_5', cnn, if_exists='replace')


query_6 = """
SELECT driverID, raceId14, raceId13, forename, surname, (fastestSeconds14 - fastestSeconds13) AS swing
FROM res_5
ORDER BY swing DESC
LIMIT 5;
"""

res_6.to_sql('res_6', cnn, if_exists='replace')




res_6 = pd.read_sql(query_6, cnn)
res_6



,driverId,raceId14,raceId13,forename,surname,swing
0,1,900,880,Lewis,Hamilton,20.001
1,817,901,880,Daniel,Ricciardo,13.998
2,13,900,880,Felipe,Massa,10.001
3,20,900,880,Sebastian,Vettel,6.997
4,824,900,880,Jules,Bianchi,5.001


# Analysis 

From above, the 5 drivers with the biggest changes in their fastestLapTime calculated in seconds across years 2014 versus 2013 by way of the regulatory changes to the sport made in 2014. 

Observations-- 
                         2013 & 2014
* Australian Grand Prix (880, 901) : Hamilton, Massa, Vettel, Bianchi
* Malaysian Grand Prix       (900) : Ricciardo

In [56]:
# import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")

#checking work in pandas
checkq = enconew[(enconew["driverId"] == 1) &  ((enconew["year"] == 2013 ) | (enconew["year"] == 2014) )]
checkq = checkq[ (checkq['raceId'] == 880) | (checkq['raceId'] == 900) ]
checkq[["raceId", "fastestLapTime", "year", "racename", "forename", "surname", "fastestSeconds"]]



,raceId,fastestLapTime,year,racename,forename,surname,fastestSeconds
110,880,01:29.8,2013,Australian Grand Prix,Lewis,Hamilton,89.008
129,900,01:49.9,2014,Australian Grand Prix,Lewis,Hamilton,109.009


The difference in Lewis Hamiltons fastestLaptime is 1:49.9 - 1:29.8 indeed a 20.001 change from 2013 to 2014 as a result of the formula change. 